In [1]:
from selenium import webdriver

# open startengine; load jquery
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get("https://www.startengine.com/explore") 
with open('jquery.js', 'r') as jquery_js:
    driver.execute_script( jquery_js.read() ) 

In [2]:
myJs = """
getStats = function ( el) {
	// result -- list of dictionaries
	result = []

	// gives statistics (amount raised, et)
	stats = $(el).find("div.stat")

	// for each of the things in stats we want the details
	stats.each( function( index, j ){
		// do stuff with j
		console.log('element', index, $(j).html() )
		// grab 'raised' or 'min invest'
		m = $(j).html().match(/<span>(.*?)<\/span/)[1]
		// amount (like $11.6M)
		n = $(j).find("div.num").text()
		console.log('m: ', m, 'n: ', n)
		var obj = {}
		obj[m] = n
		result.push ( obj  )
	})	
	project = {}
	project['name'] = $(el).find('.tombstone-content h5').text()
	project['stats'] = result
	return ( project  ) 
}


// els: list of all the projects

window.allResults = []
els.each( function(index, e) {

	window.allResults.push( getStats ( $(e) ) )
})
"""

In [3]:
driver.execute_script( myJs ) 

In [7]:
myData = driver.execute_script( 'return window.allResults' )
print('type of myData', type( myData) )
print('type of myData[0]', type( myData[0]) )
print(myData[0])


type of myData <class 'list'>
type of myData[0] <class 'dict'>
{'name': 'Terracycle', 'stats': [{'Raised': '$11.6M'}, {'Min Invest': '$700'}]}


In [ ]:
# remember that if we did regex to get to amount raised we had this regex:
# col-xs-4 col-centered stat">.*?\$(\d*).*?"abbrev-number">(\w)<.*?<span>Raised<\/span>

In [ ]:
# in console, grab divs with class tombstone: $('div.tombstone')
# notice div with class 'tombstone-stats row'
# do you see that each div within that holds 'something' (amount raised, or min invest)
# what we can do is get all of these (get the text in span, and get the 'num')

In [ ]:
Javascript, use this in the console (F12):
    
els = $('div.tombstone')
el = els[0]
$(el).find('.tombstone-stats')
$(el).find('.tombstone-stats .stat')
$(el).find('.tombstone-stats .stat .num').text()

el_1 = $(el).find('.tombstone-stats .stat')[0]
$(el_1).find("div").text()
$(el_1).html().includes('<span>Raised</span>')

el_2 = $(el).find('.tombstone-stats .stat')[1]
$(el_2).find("div").text()
$(el_1).html().includes('<span>Min Invest</span>')

/* function to figure out what is in the cell */
getVariable  = function (el){

    if (  $(el).html().includes('<span>Raised</span>') ) {
        return 'raised'
    } else if ( $(el).html().includes('<span>Min Invest</span>')  ) {
        return 'min_invest'
    } else if ( $(el).html().includes('<span>Days Left</span>')  ) {
        return 'days_left'
    } else if ( $(el).html().includes('<span>Investors</span>')  ) {
        return 'investors'
    } else {
        return 'unknown'
    }
}

/* function to get what we want from each project */
ProcessProject = function (el){
    project = {}
    project['name'] = $(el).find('.tombstone-content h5').text()
    /*project['url'] = $(el).find('a').attr('href'); */
    info = $(el).find('.tombstone-stats .stat');
    info.each( function( index, j ){
        text = $(j).find("div").text();
        varType = getVariable(j); 
        project[varType] = text        
    } )
    return (project)
}

/* example */
ProcessProject(el)

/* go through all projects */
result = []
$('div.tombstone').each( function (index, project){
    result.push ( ProcessProject (project) );
})
console.log(result)


In [2]:
mycode = """
getVariable  = function (el){

    if (  $(el).html().includes('<span>Raised</span>') ) {
        return 'raised'
    } else if ( $(el).html().includes('<span>Min Invest</span>')  ) {
        return 'min_invest'
    } else if ( $(el).html().includes('<span>Days Left</span>')  ) {
        return 'days_left'
    } else if ( $(el).html().includes('<span>Investors</span>')  ) {
        return 'investors'
    } else {
        return 'unknown'
    }
}


ProcessProject = function (el){
    project = {}
    project['name'] = $(el).find('.tombstone-content h5').text()
    /*project['url'] = $(el).find('a').attr('href'); */
    info = $(el).find('.tombstone-stats .stat');
    info.each( function( index, j ){
        text = $(j).find("div").text();
        varType = getVariable(j); 
        project[varType] = text        
    } )
    return (project)
}

window.getAllProjects = function() {
    result = []
    $('div.tombstone').each( function (index, project){
        result.push ( ProcessProject (project) );
    });
    return (result)
}


"""
driver.execute_script( mycode  )
projects = driver.execute_script( 'return window.getAllProjects()'  )
projects[0:5]

[{'min_invest': '$700', 'name': 'Terracycle', 'raised': '$11.5M'},
 {'min_invest': '$1K', 'name': 'Knightscope', 'raised': '$7.7M'},
 {'investors': '3.2K',
  'min_invest': '$500',
  'name': 'Digital Brands Group',
  'raised': '$4.9M'},
 {'investors': '4.2K',
  'min_invest': '$503',
  'name': 'StartEngine',
  'raised': '$4.9M'},
 {'min_invest': '$2.5K', 'name': 'GolfBoard', 'raised': '$1.2M'}]